### Load all files, unify station codes, groupped by trips

In [1]:
import pandas as pd
import os

In [3]:
rides = {}
base_path = "../Downloaded_data"

for year in range(2014, 2018):  # 2014 to 2017
    monthly_dfs = []
    
    for month in range(4, 12):  # April to November
        file_name = f"OD_{year}-{month:02d}.csv"
        file_path = os.path.join(base_path, str(year), file_name)
    
    # Combine monthly data into one DataFrame for the year
    if monthly_dfs:
        year_df = pd.concat(monthly_dfs, ignore_index=True)
        rides[year] = year_df
        globals()[f"rides_{year}"] = year_df  # optional: create rides_2014, rides_2015, etc.



In [4]:
for year in range(2014, 2018):
    file_path = os.path.join(base_path, str(year), f"Stations_{year}.csv")
    temp_df = pd.read_csv(file_path)
    temp_df['year'] = year
    globals()[f"stations_{year}"] = temp_df  


In [5]:
rides_2014.head(4)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,2014-04-15 00:01,6209,2014-04-15 00:18,6436,1061,1
1,2014-04-15 00:01,6214,2014-04-15 00:11,6248,615,1
2,2014-04-15 00:01,6164,2014-04-15 00:18,6216,1031,1
3,2014-04-15 00:01,6214,2014-04-15 00:24,6082,1382,1


In [6]:
stations_2014.head(2)

,code,name,latitude,longitude,year
0,6209,Milton / Clark,45.512520,-73.570620,2014
1,6436,Côte St-Antoine / Clarke,45.486452,-73.595234,2014


In [7]:
stations_2014[stations_2014.code == 6184]

,code,name,latitude,longitude,year
55,6184,Métro Mont-Royal (Rivard / du Mont-Royal),45.524673,-73.58255,2014


In [8]:
stations_2014[stations_2014.code == 6015]

,code,name,latitude,longitude,year
105,6015,Berri / de Maisonneuve,45.515299,-73.561273,2014


#### Unify code stations accross dataframes

In [9]:
stations_all = pd.concat([stations_2014, stations_2015, stations_2016, stations_2017], ignore_index=True)

unique_stations = stations_all.drop_duplicates(subset='name').reset_index(drop=True)

# Assign new unified station codes
unique_stations['unified_code'] = range(1, len(unique_stations) + 1)


In [10]:
# Drop duplicates by name 
unique_stations = stations_all.drop_duplicates(subset='name').reset_index(drop=True)

# Assign new unified station codes
unique_stations['unified_code'] = range(1, len(unique_stations) + 1)

In [11]:
# Create mapping from (yearly code -> name)
code_to_name = stations_all[['code', 'name']].drop_duplicates()

# Join with unique stations to get unified codes
code_to_unified = code_to_name.merge(unique_stations[['name', 'unified_code']], on='name', how='left')


In [12]:
code_to_unified.head(2)

,code,name,unified_code
0,6209,Milton / Clark,1
1,6436,Côte St-Antoine / Clarke,2


In [13]:
def update_ride_codes(rides_df):
    rides_df = rides_df.merge(code_to_unified.rename(columns={
        'code': 'start_station_code',
        'unified_code': 'start_unified_code'
    }), on='start_station_code', how='left')

    rides_df = rides_df.merge(code_to_unified.rename(columns={
        'code': 'end_station_code',
        'unified_code': 'end_unified_code'
    }), on='end_station_code', how='left')

    return rides_df


In [14]:
rides_2014 = update_ride_codes(rides_2014)
rides_2015 = update_ride_codes(rides_2015)
rides_2016 = update_ride_codes(rides_2016)
rides_2017 = update_ride_codes(rides_2017)

In [15]:
rides_all = pd.concat([rides_2014, rides_2015, rides_2016, rides_2017],ignore_index=True)

In [16]:
rides_all_droped = rides_all.drop(['start_station_code', 'end_station_code'], axis = 1)

In [17]:
rides_all_droped = rides_all_droped.rename(columns = {'name_x':'name_start',
                                                     'name_y': 'name_end',
                                                     'start_unified_code': 'start_station_code',
                                                     'end_unified_code': 'end_station_code'})

In [19]:
rides_all_droped.head(2)

,start_date,end_date,duration_sec,is_member,name_start,start_station_code,name_end,end_station_code
0,2014-04-15 00:01,2014-04-15 00:18,1061,1,Milton / Clark,1,Côte St-Antoine / Clarke,2
1,2014-04-15 00:01,2014-04-15 00:11,615,1,Square St-Louis,3,St-Dominique / Rachel,4


In [20]:
rides_all_droped['date'] = pd.to_datetime(rides_all['start_date']).dt.date

# Group by date, route
grouped = rides_all_droped.groupby(['date', 'start_station_code', 'end_station_code', 'name_start', 'name_end']).agg(
    duration_sec=('duration_sec', 'sum'),
    trip_count=('duration_sec', 'count'),  # count of rides (rows)
    memb_count=('is_member', 'sum')      # sum of 1s gives number of members
).reset_index()

In [21]:
grouped.head(2)

,date,start_station_code,end_station_code,name_start,name_end,duration_sec,trip_count,memb_count
0,2014-04-15,1,2,Milton / Clark,Côte St-Antoine / Clarke,1061,1,1
1,2014-04-15,1,3,Milton / Clark,Square St-Louis,173,1,1


In [24]:
grouped.to_csv('../Project_datasets/Aggregated_all_rides_codes.csv')

In [22]:
grouped = grouped.drop_duplicates()

In [23]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15592774 entries, 0 to 15592773
Data columns (total 8 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   date                object
 1   start_station_code  int64 
 2   end_station_code    int64 
 3   name_start          object
 4   name_end            object
 5   duration_sec        int64 
 6   trip_count          int64 
 7   memb_count          int64 
dtypes: int64(5), object(3)
memory usage: 951.7+ MB


In [ ]:
10735819